In [12]:
import requests
import sys
sys.path.append('/home/myntra/pushkar_playground/phi2')
from s_obj_met_agent import objective_met_agent
from question_generation_agent import question_generation_agent
from pydantic import BaseModel
from typing import List, Dict, Union, Tuple


In [9]:
def generate_response(prompt, max_new_tokens=200):
    # Set the API endpoint URL
    api_url = "http://10.12.0.67:8080/generate-response"  # Update with your actual server URL
    
    # Prepare the request payload
    prompt_data = {"prompt": prompt, "max_new_tokens" : max_new_tokens}
    
    # Send the POST request
    response = requests.post(api_url, json=prompt_data)
    if response.status_code == 200:
        # The request was successful
        generated_text = response.json()["generated_text"]
        # print(f"Generated Text: {generated_text}")
        return generated_text
    else:
        # Something went wrong
        print(f"Error: {response.status_code}, {response.text}")
        return None

In [13]:
class QuestionConfig(BaseModel):
	followup_depth: int
	criteria: List[str]

class Question(BaseModel):
	question: str
	question_config: QuestionConfig

In [ ]:
class SurveyBotV1(BaseModel):
	question_generation_agent = question_generation_agent
	objective_met_agent = objective_met_agent
	user_characteristics: Dict[str, Union[str, int, float, bool, None]] = {}
	chat_history: List[List[Tuple[int, str, str]]] = []
	fixed_questions: List[Tuple[Question, List[str]]] = []
	current_question_index: int = 0
	current_question_followup_depth: int = 0

	def get_next_question(user_answer: str):
		objective_remaining_list = question_generation_agent(chat_history, \
		fixed_questions[current_question_index].question, objectives_left)
		if len(objective_remaining_list) == 0 or current_question_followup_depth >= fixed_questions[current_question_index][0].question_config.followup_depth:
			current_question_index += 1
			current_question_follow_up_depth = 0
			if len(fixed_questions) > current_question_index:
				return fixed_questions[current_question_index]
			else:
				None
		
		current_question_follow_up_depth += 1
		return question_generation_agent(chat_history, fixed_questions[current_question_index][0].question, \
			fixed_questions[current_question_index][1], user_charecteristics)